In [1]:

import pandas as pd
from nltk.tokenize import word_tokenize

In [2]:
train = pd.read_csv('data/WikiQA-train.tsv', sep='\t')
dev = pd.read_csv('data/WikiQA-dev.tsv', sep='\t')
test = pd.read_csv('data/WikiQA-test.tsv', sep='\t')
train

,QuestionID,Question,DocumentID,DocumentTitle,SentenceID,Sentence,Label
0,Q1,how are glacier caves formed?,D1,Glacier cave,D1-0,A partly submerged glacier cave on Perito More...,0
1,Q1,how are glacier caves formed?,D1,Glacier cave,D1-1,The ice facade is approximately 60 m high,0
2,Q1,how are glacier caves formed?,D1,Glacier cave,D1-2,Ice formations in the Titlis glacier cave,0
3,Q1,how are glacier caves formed?,D1,Glacier cave,D1-3,A glacier cave is a cave formed within the ice...,1
4,Q1,how are glacier caves formed?,D1,Glacier cave,D1-4,"Glacier caves are often called ice caves , but...",0
5,Q2,How are the directions of the velocity and for...,D2,Circular motion,D2-0,"In physics , circular motion is a movement of ...",0
6,Q2,How are the directions of the velocity and for...,D2,Circular motion,D2-1,"It can be uniform, with constant angular rate ...",0
7,Q2,How are the directions of the velocity and for...,D2,Circular motion,D2-2,The rotation around a fixed axis of a three-di...,0
8,Q2,How are the directions of the velocity and for...,D2,Circular motion,D2-3,The equations of motion describe the movement ...,0
9,Q2,How are the directions of the velocity and for...,D2,Circular motion,D2-4,Examples of circular motion include: an artifi...,0


In [3]:
def preprocess(sent):
    return word_tokenize(sent.decode('utf-8').encode("ascii","ignore").lower())

train['Question_tok'] = train['Question'].map(preprocess)
train['Sentence_tok'] = train['Sentence'].map(preprocess)
dev['Question_tok'] = dev['Question'].map(preprocess)
dev['Sentence_tok'] = dev['Sentence'].map(preprocess)
test['Question_tok'] = test['Question'].map(preprocess)
test['Sentence_tok'] = test['Sentence'].map(preprocess)
train[0:5]

,QuestionID,Question,DocumentID,DocumentTitle,SentenceID,Sentence,Label,Question_tok,Sentence_tok
0,Q1,how are glacier caves formed?,D1,Glacier cave,D1-0,A partly submerged glacier cave on Perito More...,0,"[how, are, glacier, caves, formed, ?]","[a, partly, submerged, glacier, cave, on, peri..."
1,Q1,how are glacier caves formed?,D1,Glacier cave,D1-1,The ice facade is approximately 60 m high,0,"[how, are, glacier, caves, formed, ?]","[the, ice, facade, is, approximately, 60, m, h..."
2,Q1,how are glacier caves formed?,D1,Glacier cave,D1-2,Ice formations in the Titlis glacier cave,0,"[how, are, glacier, caves, formed, ?]","[ice, formations, in, the, titlis, glacier, cave]"
3,Q1,how are glacier caves formed?,D1,Glacier cave,D1-3,A glacier cave is a cave formed within the ice...,1,"[how, are, glacier, caves, formed, ?]","[a, glacier, cave, is, a, cave, formed, within..."
4,Q1,how are glacier caves formed?,D1,Glacier cave,D1-4,"Glacier caves are often called ice caves , but...",0,"[how, are, glacier, caves, formed, ?]","[glacier, caves, are, often, called, ice, cave..."


In [4]:
from itertools import chain
from gensim.models import Word2Vec

w2v = Word2Vec.load_word2vec_format('data/aquaint+wiki.txt.gz.ndim=50.bin', binary=True)
dictionary = {'PAD':0, 'UNK':1}

toks = (set(chain.from_iterable(train['Question_tok'])) | set(chain.from_iterable(train['Sentence_tok'])) | \
       set(chain.from_iterable(dev['Question_tok'])) | set(chain.from_iterable(dev['Sentence_tok']))     | \
       set(chain.from_iterable(test['Question_tok'])) | set(chain.from_iterable(test['Sentence_tok'])))

i = 2
for _, tok in enumerate(toks):
    if tok in w2v:
        dictionary[tok] = i
        i+=1
len(dictionary)

37662

In [5]:
def word2id(sent):
    return map(lambda x: dictionary.get(x, 1), sent)

train['Question_'] = train['Question_tok'].map(word2id)
train['Sentence_'] = train['Sentence_tok'].map(word2id)
dev['Question_'] = dev['Question_tok'].map(word2id)
dev['Sentence_'] = dev['Sentence_tok'].map(word2id)
test['Question_'] = test['Question_tok'].map(word2id)
test['Sentence_'] = test['Sentence_tok'].map(word2id)
train[0:5]

,QuestionID,Question,DocumentID,DocumentTitle,SentenceID,Sentence,Label,Question_tok,Sentence_tok,Question_,Sentence_
0,Q1,how are glacier caves formed?,D1,Glacier cave,D1-0,A partly submerged glacier cave on Perito More...,0,"[how, are, glacier, caves, formed, ?]","[a, partly, submerged, glacier, cave, on, peri...","[28162, 36754, 19112, 30738, 980, 23143]","[31708, 13078, 8603, 19112, 3713, 17358, 35615..."
1,Q1,how are glacier caves formed?,D1,Glacier cave,D1-1,The ice facade is approximately 60 m high,0,"[how, are, glacier, caves, formed, ?]","[the, ice, facade, is, approximately, 60, m, h...","[28162, 36754, 19112, 30738, 980, 23143]","[11943, 16722, 24896, 14267, 2527, 26639, 2356..."
2,Q1,how are glacier caves formed?,D1,Glacier cave,D1-2,Ice formations in the Titlis glacier cave,0,"[how, are, glacier, caves, formed, ?]","[ice, formations, in, the, titlis, glacier, cave]","[28162, 36754, 19112, 30738, 980, 23143]","[16722, 1897, 14278, 11943, 36141, 19112, 3713]"
3,Q1,how are glacier caves formed?,D1,Glacier cave,D1-3,A glacier cave is a cave formed within the ice...,1,"[how, are, glacier, caves, formed, ?]","[a, glacier, cave, is, a, cave, formed, within...","[28162, 36754, 19112, 30738, 980, 23143]","[31708, 19112, 3713, 14267, 31708, 3713, 980, ..."
4,Q1,how are glacier caves formed?,D1,Glacier cave,D1-4,"Glacier caves are often called ice caves , but...",0,"[how, are, glacier, caves, formed, ?]","[glacier, caves, are, often, called, ice, cave...","[28162, 36754, 19112, 30738, 980, 23143]","[19112, 30738, 36754, 34421, 37000, 16722, 307..."


In [6]:
max(len(sent) for sent in train['Question_'])

23

In [7]:
max(len(sent) for sent in train['Sentence_'])

320

In [8]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences

max_len_q = 40
max_len_a = 40

train['Question_'] = train['Question_'].apply(lambda s: pad_sequences([s], max_len_q)[0])
train['Sentence_'] = train['Sentence_'].apply(lambda s: pad_sequences([s], max_len_a)[0])
dev['Question_'] = dev['Question_'].apply(lambda s: pad_sequences([s], max_len_q)[0])
dev['Sentence_'] = dev['Sentence_'].apply(lambda s: pad_sequences([s], max_len_a)[0])
test['Question_'] = test['Question_'].apply(lambda s: pad_sequences([s], max_len_q)[0])
test['Sentence_'] = test['Sentence_'].apply(lambda s: pad_sequences([s], max_len_a)[0])
train[0:5]

Using TensorFlow backend.


,QuestionID,Question,DocumentID,DocumentTitle,SentenceID,Sentence,Label,Question_tok,Sentence_tok,Question_,Sentence_
0,Q1,how are glacier caves formed?,D1,Glacier cave,D1-0,A partly submerged glacier cave on Perito More...,0,"[how, are, glacier, caves, formed, ?]","[a, partly, submerged, glacier, cave, on, peri...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Q1,how are glacier caves formed?,D1,Glacier cave,D1-1,The ice facade is approximately 60 m high,0,"[how, are, glacier, caves, formed, ?]","[the, ice, facade, is, approximately, 60, m, h...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Q1,how are glacier caves formed?,D1,Glacier cave,D1-2,Ice formations in the Titlis glacier cave,0,"[how, are, glacier, caves, formed, ?]","[ice, formations, in, the, titlis, glacier, cave]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,Q1,how are glacier caves formed?,D1,Glacier cave,D1-3,A glacier cave is a cave formed within the ice...,1,"[how, are, glacier, caves, formed, ?]","[a, glacier, cave, is, a, cave, formed, within...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Q1,how are glacier caves formed?,D1,Glacier cave,D1-4,"Glacier caves are often called ice caves , but...",0,"[how, are, glacier, caves, formed, ?]","[glacier, caves, are, often, called, ice, cave...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [9]:
def emb_matrix(dictionary, model):
    embedding_matrix = np.zeros((len(dictionary), 50))
    for word in dictionary:
        if word in model:
            embedding_matrix[dictionary[word]] = model[word]
    return embedding_matrix

In [10]:
import numpy as np
np.random.seed(42)

from keras.models import Model, Sequential
from keras.layers import (Input,
                          Embedding,
                          Convolution1D,
                          Dropout,
                          SpatialDropout1D,
                          GlobalMaxPooling1D,
                          GlobalAveragePooling1D,
                          concatenate,
                          Dense)

from keras.optimizers import Adam


que = Input(shape=(max_len_q,))
ans = Input(shape=(max_len_a,))



que_model = Sequential()
que_model.add(Embedding(len(dictionary), 50 ,input_length=max_len_q, weights=[emb_matrix(dictionary, w2v)], trainable=True))
que_model.add(Convolution1D(100, 5, activation='tanh'))
que_model.add(GlobalAveragePooling1D())


ans_model = Sequential()
ans_model.add(Embedding(len(dictionary), 50,input_length=max_len_a, weights=[emb_matrix(dictionary, w2v)], trainable=True))
ans_model.add(Convolution1D(100, 5, activation='tanh'))
ans_model.add(GlobalAveragePooling1D())

que_emb = que_model(que)
ans_emb = ans_model(ans)

join = concatenate([que_emb, ans_emb])

classify = Sequential()
classify.add(Dense(100, activation='tanh', input_dim=200))
classify.add(Dense(1, activation='sigmoid'))
out = classify(join)

model = Model(inputs=[que, ans], outputs=[out])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 40)            0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 40)            0                                            
____________________________________________________________________________________________________
sequential_1 (Sequential)        (None, 100)           1908200     input_1[0][0]                    
____________________________________________________________________________________________________
sequential_2 (Sequential)        (None, 100)           1908200     input_2[0][0]                    
___________________________________________________________________________________________

In [11]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(0.0001),
              metrics=['accuracy'])

In [12]:
from keras.callbacks import Callback
from metrics import map_score_filtered
from sklearn.metrics import roc_auc_score

class EpochEval(Callback):

    def __init__(self, validation_data, evaluate,
                 patience=np.Inf, save_model=False, score_c=None):
        super(Callback, self).__init__()

        (self.qids, self.aids), self.X, self.y = validation_data
        self.evaluate = evaluate
        self.best = -np.Inf
        self.patience = patience
        self.wait = 0
        self.waited = False
        self.save_model = save_model


    def on_epoch_end(self, epoch, logs={}):
        print
        prediction = self.model.predict(self.X)
        val = self.evaluate(self.qids, self.y, prediction)
        print("\t{0} = {1:.4f}".format(self.evaluate.__name__, val))
        if val*0.995 > self.best:
            self.model.save('qa.h5')
            print ('\tBest {0}: {1:.4f}'.format(self.evaluate.__name__, val))
            self.best = val
            self.wait = 0
            self.waited = False
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.model.stop_training = True
        print
        
def data(dataset):
    return (dataset['QuestionID'],dataset['SentenceID']), [np.vstack(dataset['Question_'].tolist()), np.vstack(dataset['Sentence_'].tolist())], np.vstack(dataset['Label'].tolist())

In [13]:
model.fit([np.vstack(train['Question_'].tolist()), np.vstack(train['Sentence_'].tolist())],
          np.vstack(train['Label'].tolist()),
          batch_size=100,
          epochs=100000,
          shuffle=True,
          callbacks=[EpochEval(data(dev), map_score_filtered, patience=5)])

Epoch 1/100000
20300/20347 [============================>.] - ETA: 0s - loss: 0.2831 - acc: 0.9486
	map_score_filtered = 0.4057
	Best map_score_filtered: 0.4057

20347/20347 [==============================] - 23s - loss: 0.2826 - acc: 0.9487    
Epoch 2/100000
20300/20347 [============================>.] - ETA: 0s - loss: 0.2003 - acc: 0.9490
	map_score_filtered = 0.4951
	Best map_score_filtered: 0.4951

20347/20347 [==============================] - 22s - loss: 0.2004 - acc: 0.9489    
Epoch 3/100000
20300/20347 [============================>.] - ETA: 0s - loss: 0.1929 - acc: 0.9487
	map_score_filtered = 0.6167
	Best map_score_filtered: 0.6167

20347/20347 [==============================] - 21s - loss: 0.1926 - acc: 0.9488    
Epoch 4/100000
20300/20347 [============================>.] - ETA: 0s - loss: 0.1858 - acc: 0.9488
	map_score_filtered = 0.6316
	Best map_score_filtered: 0.6316

20347/20347 [==============================] - 23s - loss: 0.1858 - acc: 0.9488    
Epoch 5/100000
2

In [14]:
(qid,_ ), X, lab = data(test)

In [15]:
from keras.models import load_model
model = load_model('qa.h5')
pred = model.predict(X)

In [16]:
map_score_filtered(qid, lab, pred)

0.5855936209513423

In [17]:
from metrics import map_score

map_score(qid, lab, pred)

0.22401279785598976

In [18]:
test['pred'] = pd.Series(y for y in pred)

In [19]:
test[0:5]

,QuestionID,Question,DocumentID,DocumentTitle,SentenceID,Sentence,Label,Question_tok,Sentence_tok,Question_,Sentence_,pred
0,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-0,African immigration to the United States refer...,0,"[how, african, americans, were, immigrated, to...","[african, immigration, to, the, united, states...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0164737]
1,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-1,The term African in the scope of this article ...,0,"[how, african, americans, were, immigrated, to...","[the, term, african, in, the, scope, of, this,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0142688]
2,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-2,From the Immigration and Nationality Act of 19...,0,"[how, african, americans, were, immigrated, to...","[from, the, immigration, and, nationality, act...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[25042, 36456, 5487, 10238, 17364, 6105, 19738...",[0.0487847]
3,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-3,African immigrants in the United States come f...,0,"[how, african, americans, were, immigrated, to...","[african, immigrants, in, the, united, states,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0122861]
4,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-4,"They include people from different national, l...",0,"[how, african, americans, were, immigrated, to...","[they, include, people, from, different, natio...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0113563]


In [20]:
(qid,_ ), X, lab = data(train)
pred = model.predict(X)
print(map_score_filtered(qid, lab, pred))
print(map_score(qid, lab, pred))
train['pred'] = pd.Series(y for y in pred)
train

0.646527466936
0.266307015195


,QuestionID,Question,DocumentID,DocumentTitle,SentenceID,Sentence,Label,Question_tok,Sentence_tok,Question_,Sentence_,pred
0,Q1,how are glacier caves formed?,D1,Glacier cave,D1-0,A partly submerged glacier cave on Perito More...,0,"[how, are, glacier, caves, formed, ?]","[a, partly, submerged, glacier, cave, on, peri...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0428985]
1,Q1,how are glacier caves formed?,D1,Glacier cave,D1-1,The ice facade is approximately 60 m high,0,"[how, are, glacier, caves, formed, ?]","[the, ice, facade, is, approximately, 60, m, h...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0469075]
2,Q1,how are glacier caves formed?,D1,Glacier cave,D1-2,Ice formations in the Titlis glacier cave,0,"[how, are, glacier, caves, formed, ?]","[ice, formations, in, the, titlis, glacier, cave]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0287967]
3,Q1,how are glacier caves formed?,D1,Glacier cave,D1-3,A glacier cave is a cave formed within the ice...,1,"[how, are, glacier, caves, formed, ?]","[a, glacier, cave, is, a, cave, formed, within...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0868364]
4,Q1,how are glacier caves formed?,D1,Glacier cave,D1-4,"Glacier caves are often called ice caves , but...",0,"[how, are, glacier, caves, formed, ?]","[glacier, caves, are, often, called, ice, cave...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.023506]
5,Q2,How are the directions of the velocity and for...,D2,Circular motion,D2-0,"In physics , circular motion is a movement of ...",0,"[how, are, the, directions, of, the, velocity,...","[in, physics, ,, circular, motion, is, a, move...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0296203]
6,Q2,How are the directions of the velocity and for...,D2,Circular motion,D2-1,"It can be uniform, with constant angular rate ...",0,"[how, are, the, directions, of, the, velocity,...","[it, can, be, uniform, ,, with, constant, angu...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 142...",[0.0100079]
7,Q2,How are the directions of the velocity and for...,D2,Circular motion,D2-2,The rotation around a fixed axis of a three-di...,0,"[how, are, the, directions, of, the, velocity,...","[the, rotation, around, a, fixed, axis, of, a,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0103469]
8,Q2,How are the directions of the velocity and for...,D2,Circular motion,D2-3,The equations of motion describe the movement ...,0,"[how, are, the, directions, of, the, velocity,...","[the, equations, of, motion, describe, the, mo...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.00985646]
9,Q2,How are the directions of the velocity and for...,D2,Circular motion,D2-4,Examples of circular motion include: an artifi...,0,"[how, are, the, directions, of, the, velocity,...","[examples, of, circular, motion, include, :, a...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[29915, 19738, 31708, 22471, 36456, 14267, 102...",[0.0498558]


In [21]:
def count_feat(que, ans):
    return len((set(que)&set(ans)))

train['count'] = pd.Series(count_feat(que, ans) for que, ans in zip(train['Question_tok'], train['Sentence_tok']))
test['count'] = pd.Series(count_feat(que, ans) for que, ans in zip(test['Question_tok'], test['Sentence_tok']))
dev['count'] = pd.Series(count_feat(que, ans) for que, ans in zip(dev['Question_tok'], dev['Sentence_tok']))
train[0:5]

,QuestionID,Question,DocumentID,DocumentTitle,SentenceID,Sentence,Label,Question_tok,Sentence_tok,Question_,Sentence_,pred,count
0,Q1,how are glacier caves formed?,D1,Glacier cave,D1-0,A partly submerged glacier cave on Perito More...,0,"[how, are, glacier, caves, formed, ?]","[a, partly, submerged, glacier, cave, on, peri...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0428985],1
1,Q1,how are glacier caves formed?,D1,Glacier cave,D1-1,The ice facade is approximately 60 m high,0,"[how, are, glacier, caves, formed, ?]","[the, ice, facade, is, approximately, 60, m, h...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0469075],0
2,Q1,how are glacier caves formed?,D1,Glacier cave,D1-2,Ice formations in the Titlis glacier cave,0,"[how, are, glacier, caves, formed, ?]","[ice, formations, in, the, titlis, glacier, cave]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0287967],1
3,Q1,how are glacier caves formed?,D1,Glacier cave,D1-3,A glacier cave is a cave formed within the ice...,1,"[how, are, glacier, caves, formed, ?]","[a, glacier, cave, is, a, cave, formed, within...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0868364],2
4,Q1,how are glacier caves formed?,D1,Glacier cave,D1-4,"Glacier caves are often called ice caves , but...",0,"[how, are, glacier, caves, formed, ?]","[glacier, caves, are, often, called, ice, cave...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.023506],3


In [22]:
import numpy as np
np.random.seed(42)

from keras.models import Model, Sequential
from keras.layers import (Input,
                          Embedding,
                          Convolution1D,
                          Dropout,
                          SpatialDropout1D,
                          GlobalMaxPooling1D,
                          GlobalAveragePooling1D,
                          concatenate,
                          Dense)

from keras.optimizers import Adam


que = Input(shape=(max_len_q,))
ans = Input(shape=(max_len_a,))
cnt = Input(shape=(1,))


que_model = Sequential()
que_model.add(Embedding(len(dictionary), 50 ,input_length=max_len_q, weights=[emb_matrix(dictionary, w2v)], trainable=True))
que_model.add(Convolution1D(100, 5, activation='tanh', kernel_initializer='lecun_uniform'))
que_model.add(GlobalMaxPooling1D())


ans_model = Sequential()
ans_model.add(Embedding(len(dictionary), 50 ,input_length=max_len_a, weights=[emb_matrix(dictionary, w2v)], trainable=True))
ans_model.add(Convolution1D(100, 5, activation='tanh', kernel_initializer='lecun_uniform'))
ans_model.add(GlobalMaxPooling1D())

que_emb = que_model(que)
ans_emb = ans_model(ans)

join = concatenate([que_emb, ans_emb, cnt])

classify = Sequential()
classify.add(Dense(100, activation='tanh', input_dim=201))
classify.add(Dense(1, activation='sigmoid'))
out = classify(join)

model = Model(inputs=[que, ans, cnt], outputs=[out])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 40)            0                                            
____________________________________________________________________________________________________
input_4 (InputLayer)             (None, 40)            0                                            
____________________________________________________________________________________________________
sequential_7 (Sequential)        (None, 100)           1908200     input_3[0][0]                    
____________________________________________________________________________________________________
sequential_8 (Sequential)        (None, 100)           1908200     input_4[0][0]                    
___________________________________________________________________________________________

In [23]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(0.0001),
              metrics=['accuracy'])

In [24]:
def data(dataset):
    return (dataset['QuestionID'],dataset['SentenceID']), [np.vstack(dataset['Question_'].tolist()), np.vstack(dataset['Sentence_'].tolist()), dataset['count'].as_matrix()], np.vstack(dataset['Label'].tolist())

model.fit([np.vstack(train['Question_'].tolist()), np.vstack(train['Sentence_'].tolist()), train['count'].as_matrix()],
          np.vstack(train['Label'].tolist()),
          batch_size=100,
          epochs=100000,
          shuffle=True,
          callbacks=[EpochEval(data(dev), map_score_filtered, patience=5)])



Epoch 1/100000
20300/20347 [============================>.] - ETA: 0s - loss: 0.2590 - acc: 0.9236
	map_score_filtered = 0.5598
	Best map_score_filtered: 0.5598

20347/20347 [==============================] - 33s - loss: 0.2586 - acc: 0.9238    
Epoch 2/100000
20300/20347 [============================>.] - ETA: 0s - loss: 0.1897 - acc: 0.9490
	map_score_filtered = 0.6314
	Best map_score_filtered: 0.6314

20347/20347 [==============================] - 32s - loss: 0.1899 - acc: 0.9489    
Epoch 3/100000
20300/20347 [============================>.] - ETA: 0s - loss: 0.1791 - acc: 0.9488
	map_score_filtered = 0.7080
	Best map_score_filtered: 0.7080

20347/20347 [==============================] - 34s - loss: 0.1788 - acc: 0.9489    
Epoch 4/100000
20300/20347 [============================>.] - ETA: 0s - loss: 0.1693 - acc: 0.9489
	map_score_filtered = 0.7198
	Best map_score_filtered: 0.7198

20347/20347 [==============================] - 31s - loss: 0.1692 - acc: 0.9489    
Epoch 5/100000
2

In [25]:
del model
model = load_model('qa.h5')
(qid,_ ), X, lab = data(test)
pred = model.predict(X)
print(map_score_filtered(qid, lab, pred))
print(map_score(qid, lab, pred))
test['pred_cnt'] = pd.Series(y for y in pred)
test[0:5]

0.69107341215
0.264363003696


,QuestionID,Question,DocumentID,DocumentTitle,SentenceID,Sentence,Label,Question_tok,Sentence_tok,Question_,Sentence_,pred,count,pred_cnt
0,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-0,African immigration to the United States refer...,0,"[how, african, americans, were, immigrated, to...","[african, immigration, to, the, united, states...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0164737],4,[0.0239341]
1,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-1,The term African in the scope of this article ...,0,"[how, african, americans, were, immigrated, to...","[the, term, african, in, the, scope, of, this,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0142688],3,[0.00675429]
2,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-2,From the Immigration and Nationality Act of 19...,0,"[how, african, americans, were, immigrated, to...","[from, the, immigration, and, nationality, act...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[25042, 36456, 5487, 10238, 17364, 6105, 19738...",[0.0487847],3,[0.0431022]
3,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-3,African immigrants in the United States come f...,0,"[how, african, americans, were, immigrated, to...","[african, immigrants, in, the, united, states,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0122861],2,[0.00697259]
4,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-4,"They include people from different national, l...",0,"[how, african, americans, were, immigrated, to...","[they, include, people, from, different, natio...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0113563],0,[0.00486128]


In [26]:
def overlap(que, ans):
    return (set(que)&set(ans))

train['overlap'] = pd.Series(overlap(que, ans) for que, ans in zip(train['Question_tok'], train['Sentence_tok']))
test['overlap'] = pd.Series(overlap(que, ans) for que, ans in zip(test['Question_tok'], test['Sentence_tok']))
dev['overlap'] = pd.Series(overlap(que, ans) for que, ans in zip(dev['Question_tok'], dev['Sentence_tok']))
train[0:5]

,QuestionID,Question,DocumentID,DocumentTitle,SentenceID,Sentence,Label,Question_tok,Sentence_tok,Question_,Sentence_,pred,count,overlap
0,Q1,how are glacier caves formed?,D1,Glacier cave,D1-0,A partly submerged glacier cave on Perito More...,0,"[how, are, glacier, caves, formed, ?]","[a, partly, submerged, glacier, cave, on, peri...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0428985],1,{glacier}
1,Q1,how are glacier caves formed?,D1,Glacier cave,D1-1,The ice facade is approximately 60 m high,0,"[how, are, glacier, caves, formed, ?]","[the, ice, facade, is, approximately, 60, m, h...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0469075],0,{}
2,Q1,how are glacier caves formed?,D1,Glacier cave,D1-2,Ice formations in the Titlis glacier cave,0,"[how, are, glacier, caves, formed, ?]","[ice, formations, in, the, titlis, glacier, cave]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0287967],1,{glacier}
3,Q1,how are glacier caves formed?,D1,Glacier cave,D1-3,A glacier cave is a cave formed within the ice...,1,"[how, are, glacier, caves, formed, ?]","[a, glacier, cave, is, a, cave, formed, within...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0868364],2,"{glacier, formed}"
4,Q1,how are glacier caves formed?,D1,Glacier cave,D1-4,"Glacier caves are often called ice caves , but...",0,"[how, are, glacier, caves, formed, ?]","[glacier, caves, are, often, called, ice, cave...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.023506],3,"{glacier, caves, are}"


In [27]:
def overlap_feats(st, overlapping):
    return [1 if word not in overlapping else 2 for word in st]

train['Question_ov'] = pd.Series(overlap_feats(que, ov) for que, ov in zip(train['Question_tok'], train['overlap']))
train['Sentence_ov'] = pd.Series(overlap_feats(que, ov) for que, ov in zip(train['Sentence_tok'], train['overlap']))
dev['Question_ov'] = pd.Series(overlap_feats(que, ov) for que, ov in zip(dev['Question_tok'], dev['overlap']))
dev['Sentence_ov'] = pd.Series(overlap_feats(que, ov) for que, ov in zip(dev['Sentence_tok'], dev['overlap']))
test['Question_ov'] = pd.Series(overlap_feats(que, ov) for que, ov in zip(test['Question_tok'], test['overlap']))
test['Sentence_ov'] = pd.Series(overlap_feats(que, ov) for que, ov in zip(test['Sentence_tok'], test['overlap']))


In [28]:
train['Question_ov'] = train['Question_ov'].map(lambda s: pad_sequences([s], max_len_q)[0])
train['Sentence_ov'] = train['Sentence_ov'].map(lambda s: pad_sequences([s], max_len_a)[0])
dev['Question_ov'] = dev['Question_ov'].map(lambda s: pad_sequences([s], max_len_q)[0])
dev['Sentence_ov'] = dev['Sentence_ov'].map(lambda s: pad_sequences([s], max_len_a)[0])
test['Question_ov'] = test['Question_ov'].map(lambda s: pad_sequences([s], max_len_q)[0])
test['Sentence_ov'] = test['Sentence_ov'].map(lambda s: pad_sequences([s], max_len_a)[0])
train[0:5]

,QuestionID,Question,DocumentID,DocumentTitle,SentenceID,Sentence,Label,Question_tok,Sentence_tok,Question_,Sentence_,pred,count,overlap,Question_ov,Sentence_ov
0,Q1,how are glacier caves formed?,D1,Glacier cave,D1-0,A partly submerged glacier cave on Perito More...,0,"[how, are, glacier, caves, formed, ?]","[a, partly, submerged, glacier, cave, on, peri...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0428985],1,{glacier},"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Q1,how are glacier caves formed?,D1,Glacier cave,D1-1,The ice facade is approximately 60 m high,0,"[how, are, glacier, caves, formed, ?]","[the, ice, facade, is, approximately, 60, m, h...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0469075],0,{},"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Q1,how are glacier caves formed?,D1,Glacier cave,D1-2,Ice formations in the Titlis glacier cave,0,"[how, are, glacier, caves, formed, ?]","[ice, formations, in, the, titlis, glacier, cave]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0287967],1,{glacier},"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,Q1,how are glacier caves formed?,D1,Glacier cave,D1-3,A glacier cave is a cave formed within the ice...,1,"[how, are, glacier, caves, formed, ?]","[a, glacier, cave, is, a, cave, formed, within...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0868364],2,"{glacier, formed}","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Q1,how are glacier caves formed?,D1,Glacier cave,D1-4,"Glacier caves are often called ice caves , but...",0,"[how, are, glacier, caves, formed, ?]","[glacier, caves, are, often, called, ice, cave...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.023506],3,"{glacier, caves, are}","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [29]:
import numpy as np
np.random.seed(42)

from keras.models import Model, Sequential
from keras.layers import (Input,
                          Embedding,
                          Convolution1D,
                          Dropout,
                          SpatialDropout1D,
                          GlobalMaxPooling1D,
                          GlobalAveragePooling1D,
                          concatenate,
                          Dense)

from keras.optimizers import Adam


que = Input(shape=(max_len_q,))
ans = Input(shape=(max_len_a,))
que_ov = Input(shape=(max_len_q,))
ans_ov = Input(shape=(max_len_a,))

que_ov_emb = Embedding(3, 5,input_length=max_len_q)(que_ov)
que_word_emb = Embedding(len(dictionary), 50 , weights=[emb_matrix(dictionary, w2v)],input_length=max_len_q, trainable=True)(que)

que_emb = concatenate([que_ov_emb, que_word_emb])

ans_ov_emb = Embedding(3, 5,input_length=max_len_a)(ans_ov)
ans_word_emb = Embedding(len(dictionary), 50 , weights=[emb_matrix(dictionary, w2v)],input_length=max_len_a, trainable=True)(ans)

ans_emb = concatenate([ans_ov_emb, ans_word_emb])

que_model = Sequential()
#que_model.add(Embedding(len(dictionary), 50 , weights=[emb_matrix(dictionary, w2v)],input_length=max_len_q, trainable=True))
que_model.add(Convolution1D(100, 5, activation='tanh', kernel_initializer='lecun_uniform', input_shape=(max_len_a, 55)))
que_model.add(GlobalMaxPooling1D())


ans_model = Sequential()
#ans_model.add(Embedding(len(dictionary), 50 , weights=[emb_matrix(dictionary, w2v)],input_length=max_len_a, trainable=True))
ans_model.add(Convolution1D(100, 5, activation='tanh', kernel_initializer='lecun_uniform', input_shape=(max_len_a, 55)))
ans_model.add(GlobalMaxPooling1D())

que_emb = que_model(que_emb)
ans_emb = ans_model(ans_emb)

join = concatenate([que_emb, ans_emb])

classify = Sequential()
classify.add(Dense(100, activation='tanh', input_dim=200))
classify.add(Dense(1, activation='sigmoid'))
out = classify(join)

model = Model(inputs=[que, ans,que_ov, ans_ov], outputs=[out])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_8 (InputLayer)             (None, 40)            0                                            
____________________________________________________________________________________________________
input_6 (InputLayer)             (None, 40)            0                                            
____________________________________________________________________________________________________
input_9 (InputLayer)             (None, 40)            0                                            
____________________________________________________________________________________________________
input_7 (InputLayer)             (None, 40)            0                                            
___________________________________________________________________________________________

In [30]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(0.0001),
              metrics=['accuracy'])

In [31]:
def data(dataset):
    return (dataset['QuestionID'],dataset['SentenceID']), [np.vstack(dataset['Question_'].tolist()), np.vstack(dataset['Sentence_'].tolist()),np.vstack(dataset['Question_ov'].tolist()), np.vstack(dataset['Sentence_ov'].tolist())], np.vstack(dataset['Label'].tolist())

model.fit([np.vstack(train['Question_'].tolist()), np.vstack(train['Sentence_'].tolist()), np.vstack(train['Question_ov'].tolist()), np.vstack(train['Sentence_ov'].tolist())],
          np.vstack(train['Label'].tolist()),
          batch_size=100,
          epochs=100000,
          shuffle=True,
          callbacks=[EpochEval(data(dev), map_score_filtered, patience=5)])

Epoch 1/100000
20300/20347 [============================>.] - ETA: 0s - loss: 0.2475 - acc: 0.9480
	map_score_filtered = 0.4911
	Best map_score_filtered: 0.4911

20347/20347 [==============================] - 41s - loss: 0.2471 - acc: 0.9481    
Epoch 2/100000
20300/20347 [============================>.] - ETA: 0s - loss: 0.1940 - acc: 0.9490
	map_score_filtered = 0.5902
	Best map_score_filtered: 0.5902

20347/20347 [==============================] - 39s - loss: 0.1941 - acc: 0.9489    
Epoch 3/100000
20300/20347 [============================>.] - ETA: 0s - loss: 0.1809 - acc: 0.9490
	map_score_filtered = 0.6805
	Best map_score_filtered: 0.6805

20347/20347 [==============================] - 35s - loss: 0.1811 - acc: 0.9489    
Epoch 4/100000
20300/20347 [============================>.] - ETA: 0s - loss: 0.1688 - acc: 0.9494
	map_score_filtered = 0.6869
	Best map_score_filtered: 0.6869

20347/20347 [==============================] - 37s - loss: 0.1686 - acc: 0.9495    
Epoch 5/100000
2

In [32]:
del model
model = load_model('qa.h5')
(qid,_ ), X, lab = data(test)
pred = model.predict(X)
print(map_score_filtered(qid, lab, pred))
print(map_score(qid, lab, pred))
test['pred_ov'] = pd.Series(y for y in pred)
test[0:5]


0.692289556226
0.264828227064


,QuestionID,Question,DocumentID,DocumentTitle,SentenceID,Sentence,Label,Question_tok,Sentence_tok,Question_,Sentence_,pred,count,pred_cnt,overlap,Question_ov,Sentence_ov,pred_ov
0,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-0,African immigration to the United States refer...,0,"[how, african, americans, were, immigrated, to...","[african, immigration, to, the, united, states...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0164737],4,[0.0239341],"{to, the, were, african}","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0187274]
1,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-1,The term African in the scope of this article ...,0,"[how, african, americans, were, immigrated, to...","[the, term, african, in, the, scope, of, this,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0142688],3,[0.00675429],"{to, the, african}","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.00499742]
2,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-2,From the Immigration and Nationality Act of 19...,0,"[how, african, americans, were, immigrated, to...","[from, the, immigration, and, nationality, act...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[25042, 36456, 5487, 10238, 17364, 6105, 19738...",[0.0487847],3,[0.0431022],"{to, the, immigrated}","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, ...",[0.0368889]
3,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-3,African immigrants in the United States come f...,0,"[how, african, americans, were, immigrated, to...","[african, immigrants, in, the, united, states,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0122861],2,[0.00697259],"{the, african}","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0049105]
4,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-4,"They include people from different national, l...",0,"[how, african, americans, were, immigrated, to...","[they, include, people, from, different, natio...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0113563],0,[0.00486128],{},"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.00391548]


In [33]:
(qid,_ ), X, lab = data(train)
pred = model.predict(X)
print(map_score_filtered(qid, lab, pred))
print(map_score(qid, lab, pred))
train['pred_ov'] = pd.Series(y for y in pred)
train[0:5]

0.745143344645
0.306927253911


,QuestionID,Question,DocumentID,DocumentTitle,SentenceID,Sentence,Label,Question_tok,Sentence_tok,Question_,Sentence_,pred,count,overlap,Question_ov,Sentence_ov,pred_ov
0,Q1,how are glacier caves formed?,D1,Glacier cave,D1-0,A partly submerged glacier cave on Perito More...,0,"[how, are, glacier, caves, formed, ?]","[a, partly, submerged, glacier, cave, on, peri...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0428985],1,{glacier},"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.060285]
1,Q1,how are glacier caves formed?,D1,Glacier cave,D1-1,The ice facade is approximately 60 m high,0,"[how, are, glacier, caves, formed, ?]","[the, ice, facade, is, approximately, 60, m, h...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0469075],0,{},"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0595952]
2,Q1,how are glacier caves formed?,D1,Glacier cave,D1-2,Ice formations in the Titlis glacier cave,0,"[how, are, glacier, caves, formed, ?]","[ice, formations, in, the, titlis, glacier, cave]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0287967],1,{glacier},"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.043708]
3,Q1,how are glacier caves formed?,D1,Glacier cave,D1-3,A glacier cave is a cave formed within the ice...,1,"[how, are, glacier, caves, formed, ?]","[a, glacier, cave, is, a, cave, formed, within...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0868364],2,"{glacier, formed}","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.362595]
4,Q1,how are glacier caves formed?,D1,Glacier cave,D1-4,"Glacier caves are often called ice caves , but...",0,"[how, are, glacier, caves, formed, ?]","[glacier, caves, are, often, called, ice, cave...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.023506],3,"{glacier, caves, are}","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.11116]


In [34]:
import numpy as np
np.random.seed(42)

from keras.models import Model, Sequential
from keras.layers import (Input,
                          Embedding,
                          Convolution1D,
                          Dropout,
                          SpatialDropout1D,
                          GlobalMaxPooling1D,
                          GlobalAveragePooling1D,
                          concatenate,
                          dot,
                          Dense)

from keras.optimizers import Adam


que = Input(shape=(max_len_q,))
ans = Input(shape=(max_len_a,))
cnt = Input(shape=(1,))


que_model = Sequential()
que_model.add(Embedding(len(dictionary), 50 ,input_length=max_len_q, weights=[emb_matrix(dictionary, w2v)], trainable=True))
que_model.add(Convolution1D(100, 5, activation='tanh', kernel_initializer='lecun_uniform'))
que_model.add(GlobalMaxPooling1D())


ans_model = Sequential()
ans_model.add(Embedding(len(dictionary), 50 ,input_length=max_len_a, weights=[emb_matrix(dictionary, w2v)], trainable=True))
ans_model.add(Convolution1D(100, 5, activation='tanh', kernel_initializer='lecun_uniform'))
ans_model.add(GlobalMaxPooling1D())

que_emb = que_model(que)
ans_emb = ans_model(ans)

_que_emb = Dense(100, use_bias=False)(que_emb)
sim = dot([_que_emb, ans_emb], axes=1)

join = concatenate([que_emb, sim, ans_emb, cnt])

classify = Sequential()
classify.add(Dense(100, activation='tanh', input_dim=202))
classify.add(Dense(1, activation='sigmoid'))
out = classify(join)

model = Model(inputs=[que, ans, cnt], outputs=[out])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_10 (InputLayer)            (None, 40)            0                                            
____________________________________________________________________________________________________
sequential_19 (Sequential)       (None, 100)           1908200     input_10[0][0]                   
____________________________________________________________________________________________________
input_11 (InputLayer)            (None, 40)            0                                            
____________________________________________________________________________________________________
dense_7 (Dense)                  (None, 100)           10000       sequential_19[1][0]              
___________________________________________________________________________________________

In [35]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(0.0001),
              metrics=['accuracy'])

In [36]:
def data(dataset):
    return (dataset['QuestionID'],dataset['SentenceID']), [np.vstack(dataset['Question_'].tolist()), np.vstack(dataset['Sentence_'].tolist()), dataset['count'].as_matrix()], np.vstack(dataset['Label'].tolist())

model.fit([np.vstack(train['Question_'].tolist()), np.vstack(train['Sentence_'].tolist()), train['count'].as_matrix()],
          np.vstack(train['Label'].tolist()),
          batch_size=100,
          epochs=100000,
          shuffle=True,
          callbacks=[EpochEval(data(dev), map_score_filtered, patience=5)])

Epoch 1/100000
20300/20347 [============================>.] - ETA: 0s - loss: 0.2539 - acc: 0.9309
	map_score_filtered = 0.5589
	Best map_score_filtered: 0.5589

20347/20347 [==============================] - 40s - loss: 0.2535 - acc: 0.9310    
Epoch 2/100000
20300/20347 [============================>.] - ETA: 0s - loss: 0.1891 - acc: 0.9489
	map_score_filtered = 0.6361
	Best map_score_filtered: 0.6361

20347/20347 [==============================] - 34s - loss: 0.1891 - acc: 0.9489    
Epoch 3/100000
20300/20347 [============================>.] - ETA: 0s - loss: 0.1777 - acc: 0.9489
	map_score_filtered = 0.7062
	Best map_score_filtered: 0.7062

20347/20347 [==============================] - 28s - loss: 0.1777 - acc: 0.9489    
Epoch 4/100000
20300/20347 [============================>.] - ETA: 0s - loss: 0.1676 - acc: 0.9492
	map_score_filtered = 0.7210
	Best map_score_filtered: 0.7210

20347/20347 [==============================] - 35s - loss: 0.1675 - acc: 0.9492    
Epoch 5/100000
2

In [37]:
del model
model = load_model('qa.h5')
(qid,_ ), X, lab = data(test)
pred = model.predict(X)
print(map_score_filtered(qid, lab, pred))
print(map_score(qid, lab, pred))
test['pred_alexi'] = pd.Series(y for y in pred)
test[0:5]

0.657944191089
0.251689761988


,QuestionID,Question,DocumentID,DocumentTitle,SentenceID,Sentence,Label,Question_tok,Sentence_tok,Question_,Sentence_,pred,count,pred_cnt,overlap,Question_ov,Sentence_ov,pred_ov,pred_alexi
0,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-0,African immigration to the United States refer...,0,"[how, african, americans, were, immigrated, to...","[african, immigration, to, the, united, states...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0164737],4,[0.0239341],"{to, the, were, african}","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0187274],[0.0346864]
1,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-1,The term African in the scope of this article ...,0,"[how, african, americans, were, immigrated, to...","[the, term, african, in, the, scope, of, this,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0142688],3,[0.00675429],"{to, the, african}","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.00499742],[0.0151516]
2,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-2,From the Immigration and Nationality Act of 19...,0,"[how, african, americans, were, immigrated, to...","[from, the, immigration, and, nationality, act...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[25042, 36456, 5487, 10238, 17364, 6105, 19738...",[0.0487847],3,[0.0431022],"{to, the, immigrated}","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, ...",[0.0368889],[0.0498461]
3,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-3,African immigrants in the United States come f...,0,"[how, african, americans, were, immigrated, to...","[african, immigrants, in, the, united, states,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0122861],2,[0.00697259],"{the, african}","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0049105],[0.0176271]
4,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-4,"They include people from different national, l...",0,"[how, african, americans, were, immigrated, to...","[they, include, people, from, different, natio...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0113563],0,[0.00486128],{},"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.00391548],[0.0132211]


In [38]:
import numpy as np
np.random.seed(42)

from keras.models import Model, Sequential
from keras.layers import (Input,
                          Embedding,
                          Convolution1D,
                          Dropout,
                          SpatialDropout1D,
                          GlobalMaxPooling1D,
                          GlobalAveragePooling1D,
                          concatenate,
                          Dense)

from keras.optimizers import Adam


que = Input(shape=(max_len_q,))
ans = Input(shape=(max_len_a,))
cnt = Input(shape=(1,))


snt_model = Sequential()
snt_model.add(Embedding(len(dictionary), 50 ,input_length=max_len_q, weights=[emb_matrix(dictionary, w2v)], trainable=True))
snt_model.add(Convolution1D(100, 5, activation='tanh', kernel_initializer='lecun_uniform'))
snt_model.add(GlobalMaxPooling1D())


que_emb = snt_model(que)
ans_emb = snt_model(ans)

join = concatenate([que_emb, ans_emb, cnt])

classify = Sequential()
classify.add(Dense(100, activation='tanh', input_dim=201))
classify.add(Dense(1, activation='sigmoid'))
out = classify(join)

model = Model(inputs=[que, ans, cnt], outputs=[out])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_13 (InputLayer)            (None, 40)            0                                            
____________________________________________________________________________________________________
input_14 (InputLayer)            (None, 40)            0                                            
____________________________________________________________________________________________________
sequential_25 (Sequential)       (None, 100)           1908200     input_13[0][0]                   
                                                                   input_14[0][0]                   
____________________________________________________________________________________________________
input_15 (InputLayer)            (None, 1)             0                                   

In [39]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(0.0001),
              metrics=['accuracy'])

In [40]:
def data(dataset):
    return (dataset['QuestionID'],dataset['SentenceID']), [np.vstack(dataset['Question_'].tolist()), np.vstack(dataset['Sentence_'].tolist()), dataset['count'].as_matrix()], np.vstack(dataset['Label'].tolist())

model.fit([np.vstack(train['Question_'].tolist()), np.vstack(train['Sentence_'].tolist()), train['count'].as_matrix()],
          np.vstack(train['Label'].tolist()),
          batch_size=100,
          epochs=100000,
          shuffle=True,
          callbacks=[EpochEval(data(dev), map_score_filtered, patience=5)])

Epoch 1/100000
20300/20347 [============================>.] - ETA: 0s - loss: 0.2517 - acc: 0.9476
	map_score_filtered = 0.4669
	Best map_score_filtered: 0.4669

20347/20347 [==============================] - 29s - loss: 0.2513 - acc: 0.9478    
Epoch 2/100000
20300/20347 [============================>.] - ETA: 0s - loss: 0.1949 - acc: 0.9489
	map_score_filtered = 0.6082
	Best map_score_filtered: 0.6082

20347/20347 [==============================] - 27s - loss: 0.1947 - acc: 0.9489    
Epoch 3/100000
20300/20347 [============================>.] - ETA: 0s - loss: 0.1841 - acc: 0.9489
	map_score_filtered = 0.6597
	Best map_score_filtered: 0.6597

20347/20347 [==============================] - 26s - loss: 0.1841 - acc: 0.9489    
Epoch 4/100000
20300/20347 [============================>.] - ETA: 0s - loss: 0.1749 - acc: 0.9490
	map_score_filtered = 0.6881
	Best map_score_filtered: 0.6881

20347/20347 [==============================] - 28s - loss: 0.1749 - acc: 0.9490    
Epoch 5/100000
2

In [41]:
del model
model = load_model('qa.h5')
(qid,_ ), X, lab = data(test)
pred = model.predict(X)
print(map_score_filtered(qid, lab, pred))
print(map_score(qid, lab, pred))
test['pred_sms'] = pd.Series(y for y in pred)
test[0:5]

0.675507922866
0.258408586366


,QuestionID,Question,DocumentID,DocumentTitle,SentenceID,Sentence,Label,Question_tok,Sentence_tok,Question_,Sentence_,pred,count,pred_cnt,overlap,Question_ov,Sentence_ov,pred_ov,pred_alexi,pred_sms
0,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-0,African immigration to the United States refer...,0,"[how, african, americans, were, immigrated, to...","[african, immigration, to, the, united, states...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0164737],4,[0.0239341],"{to, the, were, african}","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0187274],[0.0346864],[0.0222085]
1,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-1,The term African in the scope of this article ...,0,"[how, african, americans, were, immigrated, to...","[the, term, african, in, the, scope, of, this,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0142688],3,[0.00675429],"{to, the, african}","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.00499742],[0.0151516],[0.0109026]
2,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-2,From the Immigration and Nationality Act of 19...,0,"[how, african, americans, were, immigrated, to...","[from, the, immigration, and, nationality, act...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[25042, 36456, 5487, 10238, 17364, 6105, 19738...",[0.0487847],3,[0.0431022],"{to, the, immigrated}","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, ...",[0.0368889],[0.0498461],[0.03684]
3,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-3,African immigrants in the United States come f...,0,"[how, african, americans, were, immigrated, to...","[african, immigrants, in, the, united, states,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0122861],2,[0.00697259],"{the, african}","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0049105],[0.0176271],[0.00991913]
4,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-4,"They include people from different national, l...",0,"[how, african, americans, were, immigrated, to...","[they, include, people, from, different, natio...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.0113563],0,[0.00486128],{},"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.00391548],[0.0132211],[0.00714233]
